In [1]:
from common import *

indents = {
    '1space': re.compile('^ \S', re.MULTILINE),
    '2space': re.compile('^ {2}\S', re.MULTILINE),
    '4space': re.compile('^ {4}\S', re.MULTILINE),
    '8space+': re.compile('^ {8}', re.MULTILINE),
    'tab': re.compile('^\t\S', re.MULTILINE),
}

indent_types = tuple(indents.keys())
indent_regexes = tuple(indents.values())

def report_indents(filename: str):
    with open(filename, 'r') as f:
        data = f.read()
    indent_counts = [sum(1 for _ in r.finditer(data)) for r in indent_regexes]
    indent_counts.append(0)  # 0 = tab, 1 = 1space, 2 = 2space
    if sum(indent_counts[:-1]) > indent_counts[-1]:
        if indent_counts[0] > 0:
            indent_counts[-1] = 1
        elif indent_counts[1] > 0:
            indent_counts[-1] = 2
        elif indent_counts[2] > 0:
            indent_counts[-1] = 4
        elif indent_counts[3] > 0:
            indent_counts[-1] = 8
    return indent_counts

In [25]:
path = os.path.join(GC_PATH, 'src/main/java/emu/grasscutter/')
results = {}
for root, dirs, files in os.walk(path):
    for filename in [os.path.join(root, f) for f in files if f.endswith('.java')]:
        key = filename[len(path):]
        results[key] = report_indents(filename)

indent_enum = {1:'1space', 2:'2space', 4:'4space', 8:'8space+', 0:'tab'}
indent_counts = [0, 0, 0, 0, 0]
indent_win_counts = {k: 0 for k in indent_enum.keys()}
for result in results.values():
    indent_win_counts[result[-1]] += 1
    for i, v in enumerate(result[:-1]):
        indent_counts[i] += v
print('Files:')
print({indent_enum[k]:v for k,v in sorted(indent_win_counts.items(), key=lambda x: x[1], reverse=True)})

print('Line instances:')
print(list(zip(indent_enum.values(), indent_counts)))

Files:
{'tab': 430, '4space': 382, '8space+': 54, '1space': 32, '2space': 0}
Line instances:
[('1space', 74), ('2space', 0), ('4space', 5685), ('8space+', 9455), ('tab', 7551)]


In [4]:
path = os.path.join(GC_PATH, 'src/main/java/emu/grasscutter/')
ending_counts = [0, 0]
for root, dirs, files in os.walk(path):
    for filename in [os.path.join(root, f) for f in files if f.endswith('.java')]:
        with open(filename, 'rb') as f:
            last_char = f.read()[-1]
            # print(last_char)
            ending_counts[int(last_char == 10)] += 1

print(f'{ending_counts[0]} without newline at EOF, {ending_counts[1]} with.')

47 without newline at EOF, 903 with.
